<a href="https://colab.research.google.com/github/thamyrissacamoto/gov-sim/blob/main/etl_dm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
from datetime import datetime
import pandas as pd
import pytz
import zipfile
import os

brasil_tz = pytz.timezone('America/Sao_Paulo')

In [ ]:
# zip_path = "mortalidade.zip"

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall()

In [ ]:
def criar_tabelas_dm():
  tabelas_sql = [
      """CREATE TABLE IF NOT EXISTS DIME_FAIXA_ETARIA (
          SK_Faixa_Etaria INTEGER PRIMARY KEY,
          CD_Faixa_Etaria INTEGER,
          DS_Faixa_Etaria TEXT,
          DT_Carga TIMESTAMP
      );""",
      """CREATE TABLE IF NOT EXISTS DIME_ESTADO_CIVIL (
          SK_Estado_Civil INTEGER PRIMARY KEY,
          CD_Estado_Civil INTEGER,
          DS_Estado_Civil TEXT,
          DT_Carga TIMESTAMP
      );""",
      """CREATE TABLE IF NOT EXISTS DIME_SEXO (
          SK_Sexo INTEGER PRIMARY KEY,
          CD_Sexo TEXT,
          DS_Sexo TEXT,
          DT_Carga TIMESTAMP
      );""",
      """CREATE TABLE IF NOT EXISTS DIME_RACA (
          SK_Raca INTEGER PRIMARY KEY,
          CD_Raca INTEGER,
          DS_Raca TEXT,
          DT_Carga TIMESTAMP
      );""",
      """CREATE TABLE IF NOT EXISTS DIME_OCUPACAO (
          SK_Ocupacao INTEGER PRIMARY KEY,
          CD_Ocupacao INTEGER,
          DS_Ocupacao TEXT,
          DT_Carga TIMESTAMP
      );""",
      """CREATE TABLE IF NOT EXISTS DIME_LOCAL (
          SK_Local INTEGER PRIMARY KEY,
          CD_UF INTEGER,
          NM_UF TEXT,
          CD_Municipio INTEGER,
          NM_Municipio TEXT,
          DT_Carga TIMESTAMP
      );""",
      """CREATE TABLE IF NOT EXISTS DIME_CID (
          SK_CID INTEGER PRIMARY KEY,
          CD_CID TEXT,
          DS_CID TEXT,
          DT_Carga TIMESTAMP
      );""",
      """CREATE TABLE IF NOT EXISTS DIME_ESCOLARIDADE (
          SK_Escolaridade INTEGER PRIMARY KEY,
          CD_Escolaridade INTEGER,
          DS_Escolaridade TEXT,
          DT_Carga TIMESTAMP
      );""",
      """CREATE TABLE IF NOT EXISTS DIME_TEMPO (
          SK_Tempo INTEGER PRIMARY KEY,
          NR_Mes INTEGER,
          NR_Ano INTEGER,
          DT_Data DATE
      );""",
      """CREATE TABLE IF NOT EXISTS FATO_OBITO (
          SK_Raca INTEGER NOT NULL,
          SK_Sexo INTEGER NOT NULL,
          SK_CID INTEGER NOT NULL,
          SK_DT_Obito INTEGER NOT NULL,
          SK_Escolaridade INTEGER NOT NULL,
          SK_Faixa_Etaria INTEGER NOT NULL,
          SK_Estado_Civil INTEGER NOT NULL,
          SK_Ocupacao INTEGER NOT NULL,
          SK_Local_Obito INTEGER NOT NULL,
          NR_Assistencia_Medica INTEGER NOT NULL,
          NR_Exame INTEGER NOT NULL,
          NR_Cirurgia INTEGER NOT NULL,
          NR_Quantidade INTEGER NOT NULL,
          DT_Carga TIMESTAMP NOT NULL,
          PRIMARY KEY(SK_Raca, SK_Sexo, SK_CID, SK_DT_Obito, SK_Escolaridade, SK_Faixa_Etaria, SK_Estado_Civil, SK_Ocupacao, SK_Local_Obito),
          FOREIGN KEY(SK_DT_Obito) REFERENCES DIME_TEMPO(SK_Tempo),
          FOREIGN KEY(SK_Raca) REFERENCES DIME_RACA(SK_Raca),
          FOREIGN KEY(SK_Sexo) REFERENCES DIME_SEXO(SK_Sexo),
          FOREIGN KEY(SK_Estado_Civil) REFERENCES DIME_ESTADO_CIVIL(SK_Estado_Civil),
          FOREIGN KEY(SK_Faixa_Etaria) REFERENCES DIME_FAIXA_ETARIA(SK_Faixa_Etaria),
          FOREIGN KEY(SK_Local_Obito) REFERENCES DIME_LOCAL(SK_Local),
          FOREIGN KEY(SK_Escolaridade) REFERENCES DIME_ESCOLARIDADE(SK_Escolaridade)
      );"""
  ]
  with sqlite3.connect(dm_file) as con_dm:
    cursor = con_dm.cursor()
    for sql in tabelas_sql:
        cursor.execute(sql)
    con_dm.commit()
    print("Dimensões criadas")

In [ ]:
def criar_dime_local():
  agora = datetime.now(brasil_tz).strftime('%Y-%m-%d %H:%M:%S')

  with sqlite3.connect(dm_file) as con_dm, sqlite3.connect(dw_file) as con_dw:
    cursor_dm = con_dm.cursor()
    cursor_dw = con_dw.cursor()

    cursor_dw.execute("""
          SELECT DISTINCT u.CD_UF, u.NM_UF, m.CD_MUNICIPIO, m.NM_MUNICIPIO
          FROM DWCD_UF u
          JOIN DWCD_MUNICIPIO m ON u.SK_UF = m.SK_UF
    """)
    rows = cursor_dw.fetchall()

    dados = []

    for i, r in enumerate(rows):
      cd_uf, nm_uf, cd_mun, nm_mun = r
      if cd_uf == -1 or cd_mun == -1:
          sk_local = -1
      else:
          sk_local = i + 1
      dados.append((sk_local, cd_uf, nm_uf.lower(), cd_mun, nm_mun.lower(), agora))

    cursor_dm.execute("DELETE FROM DIME_LOCAL")
    cursor_dm.executemany(
        "INSERT INTO DIME_LOCAL (SK_Local, CD_UF, NM_UF, CD_Municipio, NM_Municipio, DT_Carga) VALUES (?,?,?,?,?,?)",
        dados
    )
    con_dm.commit()
    print(f"{len(rows)} linhas carregadas em DIME_LOCAL")

In [ ]:
def criar_faixa_etaria():
  agora = datetime.now(brasil_tz).strftime('%Y-%m-%d %H:%M:%S')

  faixa = [
      (1, 0,'0-9 anos', agora),
      (2, 10,'10-19 anos', agora),
      (3, 20,'20-29 anos', agora),
      (4, 30,'30-39 anos', agora),
      (5, 40,'40-49 anos', agora),
      (6, 50,'50-59 anos', agora),
      (7, 60,'60-69 anos', agora),
      (8, 70,'70-79 anos', agora),
      (9, 80,'80-89 anos', agora),
      (10, 90,'90+ anos', agora)
  ]
  with sqlite3.connect(dm_file) as con_dm:
    cursor = con_dm.cursor()
    cursor.executemany(
        "INSERT OR IGNORE INTO DIME_FAIXA_ETARIA (SK_Faixa_Etaria, CD_Faixa_Etaria, DS_Faixa_Etaria, DT_Carga) VALUES (?,?,?,?)",
        faixa
    )
    con_dm.commit()
    print("DIME_FAIXA_ETARIA criada")

In [ ]:
def criar_dime_tempo():
  dados_tempo = []
  for ano in range(2018, 2025):
      for mes in range(1, 13):
          sk_tempo = int(f"{mes:02d}{ano}")
          dt_data = f"{ano}-{mes:02d}-01"
          dados_tempo.append((sk_tempo, mes, ano, dt_data))

  with sqlite3.connect(dm_file) as con_dm:
    cursor = con_dm.cursor()
    cursor.executemany(
        "INSERT OR IGNORE INTO DIME_TEMPO (SK_Tempo, NR_Mes, NR_Ano, DT_Data) VALUES (?,?,?,?)",
        dados_tempo
    )
    con_dm.commit()
    print("DIME_TEMPO criada")

In [ ]:
def carregar_dim(nome_dim, tabela_dw, batch_size=500):
  agora = datetime.now(brasil_tz).strftime('%Y-%m-%d %H:%M:%S')

  try:
    with sqlite3.connect(dm_file, timeout=60) as con_dm, sqlite3.connect(dw_file, timeout=60) as con_dw:
        cursor_dm = con_dm.cursor()
        cursor_dw = con_dw.cursor()

        cursor_dw.execute(f'SELECT * FROM "{tabela_dw}"')
        rows = cursor_dw.fetchall()
        qtd_origem = len(rows)
        print(f"\nExtraindo {qtd_origem} linhas de {tabela_dw}")

        if qtd_origem == 0:
            print(f"Nenhuma linha encontrada em {tabela_dw}")
            return
        cursor_dm.execute(f'DELETE FROM "{nome_dim}"')
        con_dm.commit()

        num_cols = len(rows[0])
        placeholders = ",".join(["?"] * num_cols)
        rows_carga = []
        for r in rows:
            r_limpo = []
            for i, val in enumerate(r[:-1]):
              if i == 0:
                  r_limpo.append(val)
              else:
                  if val is None:
                      r_limpo.append(-1)
                  elif isinstance(val, str):
                      r_limpo.append(val.strip().lower())
                  else:
                      r_limpo.append(val)


            r_limpo.append(agora)
            rows_carga.append(tuple(r_limpo))

        for i in range(0, len(rows_carga), batch_size):
            batch = rows_carga[i:i+batch_size]
            insert_sql = f'INSERT INTO "{nome_dim}" VALUES ({placeholders})'
            cursor_dm.executemany(insert_sql, batch)
            con_dm.commit()

        print(f"{qtd_origem} linhas inseridas em {nome_dim}")

  except sqlite3.OperationalError as e:
      print(f"Erro carregando {nome_dim}: {e}")

In [ ]:
def calcular_idade(dt_nasc, dt_obito):
  dt_nasc_str = str(dt_nasc).zfill(8)
  dt_obito_str = str(dt_obito).zfill(8)

  ano_nasc = int(dt_nasc_str[-4:])
  ano_obito = int(dt_obito_str[-4:])

  idade = ano_obito - ano_nasc
  return idade

In [ ]:
def calcular_faixa_etaria(idade):
  if idade is None:
    return -1

  idade = int(idade)
  if idade < 10:
    return 1
  elif idade < 20:
    return 2
  elif idade < 30:
    return 3
  elif idade < 40:
    return 4
  elif idade < 50:
    return 5
  elif idade < 60:
    return 6
  elif idade < 70:
    return 7
  elif idade < 80:
    return 8
  elif idade < 90:
    return 9
  else:
    return 10

In [ ]:
def carregar_fato_obito(full_load=True, data_inicio=None):
  agora = datetime.now(brasil_tz).strftime('%Y-%m-%d %H:%M:%S')

  with sqlite3.connect(dw_file) as conn_dw, sqlite3.connect(dm_file) as conn_dm:
      cursor_dw = conn_dw.cursor()
      cursor_dm = conn_dm.cursor()

      sql_dw = """
          SELECT
              o.SK_Dados_Demograficos,
              o.SK_CID,
              c.CD_CID,
              o.DT_Data,
              d.DT_Nascimento,
              d.SK_Sexo,
              d.SK_Raca,
              d.SK_Escolaridade,
              d.SK_Estado_Civil,
              d.SK_Ocupacao,
              o.SK_Municipio AS SK_Local_Obito,
              o.ST_Assistencia_Medica,
              o.ST_Exame,
              o.ST_Cirurgia
          FROM DWMV_OBITO o
          JOIN DWCD_DADOS_DEMOGRAFICOS d
            ON o.SK_Dados_Demograficos = d.SK_Dados_Demograficos
          JOIN DWCD_CID c
            ON o.SK_CID = c.SK_CID
          WHERE c.CD_CID BETWEEN 'A000' AND 'R99'
      """
      if data_inicio and not full_load:
          sql_dw += f" AND o.DT_Data >= '{data_inicio}'"

      cursor_dw.execute(sql_dw)
      rows_dw = cursor_dw.fetchall()
      print(f"\nTotal linhas DWMV_OBITO filtradas: {len(rows_dw)}")

      cursor_dm.execute("SELECT SK_Tempo FROM DIME_TEMPO")
      tempo_rows = cursor_dm.fetchall()
      sk_tempo_map = {str(r[0]).zfill(6): r[0] for r in tempo_rows}

      cursor_dm.execute("SELECT SK_Local, CD_UF, CD_Municipio FROM DIME_LOCAL")
      local_rows = cursor_dm.fetchall()
      dim_local = {(r[1], r[2]): r[0] for r in local_rows}

      cursor_dw.execute("SELECT SK_Municipio, CD_Municipio, SK_UF FROM DWCD_MUNICIPIO")
      muni_rows = cursor_dw.fetchall()
      cursor_dw.execute("SELECT SK_UF, CD_UF FROM DWCD_UF")
      uf_rows = cursor_dw.fetchall()
      uf_map = {r[0]: r[1] for r in uf_rows}
      muni_map = {r[0]: (r[1], uf_map.get(r[2], None)) for r in muni_rows}

      fato_dict = {}
      erros_tempo = 0
      for row in rows_dw:
          (sk_dados_demograficos, sk_cid, cd_cid, dt_data, dt_nasc,
            sk_sexo, sk_raca, sk_escolaridade, sk_estado_civil, sk_ocupacao,
            sk_local_obito, st_assistencia, st_exame, st_cirurgia) = row

          idade = calcular_idade(dt_nasc, dt_data)
          sk_faixa_etaria = calcular_faixa_etaria(idade)

          dt_data_dw = str(dt_data).zfill(8)[2:8]
          sk_dt_obito = sk_tempo_map.get(dt_data_dw, -1)

          if sk_dt_obito == -1:
              erros_tempo += 1

          muni = muni_map.get(sk_local_obito, (None, None))
          sk_local = dim_local.get((muni[1], muni[0]), -1)

          key = (
              sk_raca, sk_sexo, sk_cid, sk_dt_obito,
              sk_escolaridade, sk_ocupacao, sk_local,
              sk_estado_civil, sk_faixa_etaria
          )

          if key not in fato_dict:
            fato_dict[key] = {
                'NR_Assistencia_Medica': 1 if st_assistencia == 1 else 0,
                'NR_Exame': 1 if st_exame == 1 else 0,
                'NR_Cirurgia': 1 if st_cirurgia == 1 else 0,
                'NR_Quantidade': 1,
                'DT_Carga': agora
                }
          else:
            if st_assistencia == 1:
                fato_dict[key]['NR_Assistencia_Medica'] += 1
            if st_exame == 1:
                fato_dict[key]['NR_Exame'] += 1
            if st_cirurgia == 1:
                fato_dict[key]['NR_Cirurgia'] += 1
            fato_dict[key]['NR_Quantidade'] += 1

      print(f"\nTotal de registros sem SK_DT_Obito: {erros_tempo}")

      if full_load:
          cursor_dm.execute("DELETE FROM FATO_OBITO")

      insert_sql = """
          INSERT INTO FATO_OBITO (
              SK_Raca, SK_Sexo, SK_CID, SK_DT_Obito, SK_Escolaridade,
              SK_Ocupacao, SK_Local_Obito, SK_Estado_Civil, SK_Faixa_Etaria,
              NR_Assistencia_Medica, NR_Exame, NR_Cirurgia, NR_Quantidade, DT_Carga
          ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)
      """
      cursor_dm.executemany(
          insert_sql,
          [k + (v['NR_Assistencia_Medica'], v['NR_Exame'], v['NR_Cirurgia'], v['NR_Quantidade'], v['DT_Carga'])
            for k, v in fato_dict.items()]
      )

      conn_dm.commit()
      print(f"\nFATO_OBITO carregado com {len(fato_dict)} linhas")

In [ ]:
def exportar_dm_para_csv(dm_file):
  with sqlite3.connect(dm_file) as con:
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tabelas = [t[0] for t in cur.fetchall()]

    for tabela in tabelas:
      df = pd.read_sql_query(f'SELECT * FROM "{tabela}"', con)
      df.columns = [c.lower() for c in df.columns]
      nome_csv = f"{tabela}.csv"
      df.to_csv(nome_csv, sep=';', index=False, encoding='utf-8-sig')
      print(f"\n{nome_csv} exportado ({len(df)} linhas)")

In [ ]:
dw_file = "mortalidade.db"
dm_file = "mortalidade_dm.db"

criar_tabelas_dm()
criar_faixa_etaria()
criar_dime_tempo()
criar_dime_local()

carregar_dim("DIME_SEXO", "DWCD_SEXO")
carregar_dim("DIME_ESTADO_CIVIL", "DWCD_ESTADO_CIVIL")
carregar_dim("DIME_RACA", "DWCD_RACA")
carregar_dim("DIME_OCUPACAO", "DWCD_OCUPACAO")
carregar_dim("DIME_CID", "DWCD_CID")
carregar_dim("DIME_ESCOLARIDADE", "DWCD_ESCOLARIDADE")

carregar_fato_obito(full_load=True)

exportar_dm_para_csv("mortalidade_dm.db")

Dimensões criadas
DIME_FAIXA_ETARIA criada
DIME_TEMPO criada
5572 linhas carregadas em DIME_LOCAL

Extraindo 8 linhas de DWCD_SEXO
8 linhas inseridas em DIME_SEXO

Extraindo 7 linhas de DWCD_ESTADO_CIVIL
7 linhas inseridas em DIME_ESTADO_CIVIL

Extraindo 6 linhas de DWCD_RACA
6 linhas inseridas em DIME_RACA

Extraindo 2695 linhas de DWCD_OCUPACAO
2695 linhas inseridas em DIME_OCUPACAO

Extraindo 12452 linhas de DWCD_CID
12452 linhas inseridas em DIME_CID

Extraindo 8 linhas de DWCD_ESCOLARIDADE
8 linhas inseridas em DIME_ESCOLARIDADE

Total linhas DWMV_OBITO filtradas: 8890000

Total de registros sem SK_DT_Obito: 0

FATO_OBITO carregado com 8473119 linhas

DIME_FAIXA_ETARIA.csv exportado (10 linhas)

DIME_ESTADO_CIVIL.csv exportado (7 linhas)

DIME_SEXO.csv exportado (8 linhas)

DIME_RACA.csv exportado (6 linhas)

DIME_OCUPACAO.csv exportado (2695 linhas)

DIME_LOCAL.csv exportado (5572 linhas)

DIME_CID.csv exportado (12452 linhas)

DIME_ESCOLARIDADE.csv exportado (8 linhas)

DIME_TEM